# 04. Faissを使った本格的なRAGの実装

`03_rag_concept_demo` では、手動でコンテキストを与えてRAGの概念を学びました。
このノートブックでは、いよいよ本格的なRAGシステムを構築します。

具体的には、以下のライブラリを使って「検索」ステップを自動化します。

- **Sentence-Transformers**: 文章をベクトル（数値の配列）に変換（Embedding）するためのライブラリ。文章の意味が似ているほど、ベクトル空間上での距離が近くなります。
- **Faiss**: Facebook AIが開発した、巨大なベクトルデータの中から高速に類似ベクトルを検索するためのライブラリ。

## 事前準備

Google Colabで実行する場合、メニューの「ランタイム」→「ランタイムのタイプを変更」で、ハードウェアアクセラレータが「T4 GPU」になっていることを確認してください。

In [ ]:
# 編集禁止セル
# セットアップの確認と共通モジュールのインポート
import os
import sys
import torch
import json
from google.colab import drive

if not os.path.isdir('/content/drive'): drive.mount('/content/drive')
repo_path = '/content/llm_lab'
if not os.path.exists(repo_path):
    !git clone -b stable-base https://github.com/akio-kobayashi/llm_lab.git
os.chdir(repo_path)

# 必要なライブラリのインストール
!pip install -q -U transformers accelerate bitsandbytes sentence-transformers faiss-cpu peft datasets gradio
if 'src' not in sys.path: sys.path.append(os.path.abspath('src'))

try:
    from src.common import load_llm, generate_text
    from src.rag import FaissRAGPipeline
    print('共通モジュールのインポートが完了しました。')
except ImportError:
    print('共通モジュールのインポートに失敗しました。')

## Faissインデックスの準備

RAGで高速に検索を行うためには、あらかじめ知識源となるドキュメントをすべてベクトル化し、Faissインデックスを構築しておく必要があります。

ここでは、`data/docs/anime_docs_sample.jsonl` の内容からインデックスを構築します。
（通常は事前に構築済みのインデックスを読み込みますが、今回は学習のため構築プロセスを実行します）

In [ ]:
# 編集禁止セル
# インデックスとメタデータの保存先パスをGoogle Drive上に設定
DRIVE_DIR = '/content/drive/MyDrive/llm_lab_outputs'
INDEX_DIR = os.path.join(DRIVE_DIR, 'faiss_index')
INDEX_PATH = os.path.join(INDEX_DIR, 'anime_docs.index')
META_PATH = os.path.join(INDEX_DIR, 'anime_docs_meta.json')
DOCS_PATH = 'data/docs/anime_docs_sample.jsonl'

os.makedirs(INDEX_DIR, exist_ok=True)

# RAGパイプラインの初期化
rag_pipeline = FaissRAGPipeline()

# インデックスがまだ存在しない場合のみ、構築・保存する
if not os.path.exists(INDEX_PATH):
    print('インデックスが存在しないため、新規に構築します。')
    rag_pipeline.build_index(DOCS_PATH)
    rag_pipeline.save_index(INDEX_PATH, META_PATH)
else:
    print('既存のインデックスを読み込みます。')
    rag_pipeline.load_index(INDEX_PATH, META_PATH)

## モデルのロード

回答生成用のLLMをロードします。

In [ ]:
# 編集禁止セル
model, tokenizer = None, None
try:
    model, tokenizer = load_llm(use_4bit=True)
except Exception as e:
    print(f'モデルのロード中にエラーが発生しました: {e}')

## RAGパイプラインの実行

構築したRAGパイプラインを使って、質問応答を試してみましょう。
以下の `run_rag_pipeline` 関数は、一連のRAG処理（検索→拡張→生成）をまとめて実行します。

In [ ]:
# 編集禁止セル
def run_rag_pipeline(question, top_k=3):
    if not all([rag_pipeline, model, tokenizer]):
        print("RAGパイプラインまたはモデルが初期化されていません。")
        return

    # 1. 検索 (Retrieve)
    context_docs = rag_pipeline.search(question, top_k=top_k)
    
    print("\n--- 検索結果 (上位" + str(top_k) + "件) ---")
    for i, doc in enumerate(context_docs):
        print(f"[{i+1}] Title: {doc['title']}, Section: {doc['section']}")
        print(f"   Text: {doc['text'][:100]}...") # 長すぎるので先頭のみ表示
    
    # 2. 拡張 (Augment)
    prompt = rag_pipeline.create_prompt_with_context(question, context_docs)
    
    # 3. 生成 (Generate)
    generated_text = generate_text(model, tokenizer, prompt)
    answer = generated_text.split("回答:")[-1].strip()
    
    print("\n--- LLMによる回答 ---")
    print(answer)

### 演習1: RAGによる質問応答

まずは基本的な質問をしてみましょう。`anime_docs_sample.jsonl` に含まれる情報に基づいて、正しく回答できるはずです。

In [ ]:
# --- ここを編集 --- #
question = "『東京サイバーパンク2042』の主人公はどんな人物ですか？"
# --- 編集ここまで --- #

# 編集禁止セル
run_rag_pipeline(question)

### 演習2: `top_k` の値を変更する

検索するドキュメントの数 (`top_k`) を変えると、LLMに与えるコンテキストの量が変わります。
`top_k` を増やすと、より多くの情報を参考にできますが、ノイズ（関係ない情報）が増える可能性もあります。

同じ質問で `top_k` を `1` と `5` に変えて、検索結果と最終的な回答がどう変わるか比較してみましょう。

In [ ]:
# --- ここを編集 --- #
question = "主人公が料理をするアニメについて教えてください。"
top_k_value = 1 # この値を 5 に変えてみましょう
# --- 編集ここまで --- #

# 編集禁止セル
run_rag_pipeline(question, top_k=top_k_value)

### 演習3: 質問の表現を変える

質問の仕方を少し変えるだけで、検索結果（ベクトルの類似度）が変わり、回答に影響することがあります。
同じ意図の質問を、異なる表現で試してみましょう。

In [ ]:
# --- ここを編集 --- #
# 試行1: 直接的な質問
question_1 = "『星屑のメモリー』のあらすじを教えて。"

# 試行2: 少し曖昧な質問
question_2 = "カイ・ミナトが活躍する物語はどんな話？"
# --- 編集ここまで --- #

# 編集禁止セル
print("--- 試行1 --- ")
run_rag_pipeline(question_1, top_k=2)
print("\n" + "="*50 + "\n")
print("--- 試行2 --- ")
run_rag_pipeline(question_2, top_k=2)

### 演習4: 失敗例（曖昧な質問や知識ベースにない質問）

RAGは万能ではありません。質問が曖昧すぎたり、知識ベースに関連情報が全く存在しない場合は、うまく機能しません。

- **曖昧な質問**: 「面白いアニメは？」のような主観的な質問では、どのドキュメントを検索すればよいか特定できません。
- **知識ベースにない質問**: 「『星屑のメモリー』の続編は？」のように、`anime_docs_sample.jsonl` に含まれていない情報を質問しても、答えることはできません。

これらの「失敗例」を試してみましょう。

In [ ]:
# --- ここを編集 --- #
question = "感動的な物語が見たいです。おすすめはありますか？"
# question = "『星屑のメモリー』のBlu-rayはいつ発売されますか？" # こちらも試してみましょう
# --- 編集ここまで --- #

# 編集禁止セル
run_rag_pipeline(question)

## まとめ

このノートブックでは、Faissを使って本格的なRAGシステムを構築し、その挙動を観察しました。

- RAGは、LLMに外部の事実情報を提供し、ハルシネーションを抑制するのに非常に効果的です。
- 検索の質（`top_k`の調整、質問の仕方）が、最終的な回答の質に大きく影響します。
- RAGは、あくまで提供された知識ベースの範囲内でしか回答できません。

RAGは「事実に基づいた回答」を生成するのに役立ちますが、LLMの「応答スタイル」や「特定のタスクをこなす能力」そのものを変えるわけではありません。
例えば、「必ずJSON形式で答えてほしい」といった要求に応えさせるには、プロンプトの工夫だけでは限界があります。

この次は、`06_integrate_gradio.ipynb` でRAGをGradio UIに統合し、対話的に検証します。

### メモリ解放


In [ ]:
# 編集禁止セル
import gc
if 'model' in locals() and model is not None: del model
if 'tokenizer' in locals() and tokenizer is not None: del tokenizer
if 'rag_pipeline' in locals() and rag_pipeline is not None: del rag_pipeline
model, tokenizer, rag_pipeline = None, None, None
gc.collect()
torch.cuda.empty_cache()
print("リソースを解放し、GPUキャッシュをクリアしました。")